In [1]:
import numpy as np
import matplotlib.pyplot as plt
import importlib
import ipywidgets as iw
import dronesim
from pid import PID
import good_controller as cont

In [2]:
def simulate(cd, target_x, target_y): # simulate a ControlledDrone instance...
    dt = 0.03 # timestep
    ts = np.arange(0,10,dt)
    xs,ys,thetas,lts,rts = [],[],[],[],[] # save here the x,y,theta coordinates of the drone
    for t in ts:
        cd.step(dt, target_x, target_y)
        x,y = cd.drone.getxy()
        xs.append(x)
        ys.append(y)
        lts.append(cd.drone.lt)
        rts.append(cd.drone.rt)
        thetas.append(cd.drone.gettheta())
        print(cd.controller.tt, cd.controller.tdiff)
    fig,axs = plt.subplots(nrows=4, sharex=True, figsize=(8,12))
    axs[0].plot(ts,xs)
    axs[0].axhline(target_x, color='red')
    axs[0].set(ylabel="x position [m]")
    axs[1].plot(ts,ys)
    axs[1].axhline(target_y, color='red')
    axs[1].set(ylabel="height [m]")
    axs[2].plot(ts,np.rad2deg(thetas))
    axs[2].set(ylabel="theta [deg]")
    axs[3].plot(ts,lts)
    axs[3].plot(ts,rts)
    axs[3].set(ylabel="thrust [N]", xlabel = "time [s]")

In [3]:
importlib.reload(cont) # make sure we have the latest version if it was edited
initial_pose = dronesim.mktr(0,0) @ dronesim.mkrot(0)
d = dronesim.Drone2D(initial_pose=initial_pose, mass=1, L=1, maxthrust=1000)
c = cont.Controller(maxthrust=d.maxthrust)
cd = dronesim.ControlledDrone(drone=d, controller=c)
simulate(cd, target_x=0, target_y=10)

AttributeError: 'Controller' object has no attribute 'tt'